<a href="https://colab.research.google.com/github/OneFineStarstuff/Cosmic-Brilliance/blob/main/Complete_Enhanced_Unified_AI_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Install required libraries
# !pip install transformers torch pillow numpy gym requests

import os
import requests
import random
import numpy as np

# -------------------------------------------------------------------
# Patch: Alias np.bool8 to avoid AttributeError in Gym's checker
# -------------------------------------------------------------------
if not hasattr(np, "bool8"):
    np.bool8 = np.bool_

from io import BytesIO
from collections import deque
from PIL import Image
from transformers import pipeline
import gym

# ------------------------------------------------------------------------------
# 1. NLP: Question-Answering Module
# ------------------------------------------------------------------------------
class QAModule:
    def __init__(self, model_name="distilbert-base-cased-distilled-squad"):
        self.qa_pipeline = pipeline(
            "question-answering", model=model_name, device=-1
        )

    def answer_questions(self, context, questions):
        answers = {}
        for q in questions:
            try:
                res = self.qa_pipeline(question=q, context=context)
                if isinstance(res, dict) and "answer" in res:
                    answers[q] = res["answer"]
                else:
                    answers[q] = str(res)
            except Exception as e:
                answers[q] = f"[Error: {e}]"
        return answers

# ------------------------------------------------------------------------------
# 2. Vision: Image Captioning with Caching & Cross-Modal Retrieval
# ------------------------------------------------------------------------------
class VisionModule:
    def __init__(self, model_name="Salesforce/blip-image-captioning-base", cache_dir="image_cache"):
        self.caption_pipeline = pipeline(
            "image-to-text", model=model_name, device=-1
        )
        os.makedirs(cache_dir, exist_ok=True)
        self.cache_dir = cache_dir
        self.headers = {
            "User-Agent": "MyUnifiedAI/1.0 (kyaw@example.com) Python-requests"
        }

    def _cache_path(self, url):
        return os.path.join(self.cache_dir, url.split("/")[-1])

    def load_image(self, url):
        path = self._cache_path(url)
        if not os.path.exists(path):
            resp = requests.get(url, headers=self.headers)
            resp.raise_for_status()
            with open(path, "wb") as f:
                f.write(resp.content)
        return Image.open(path).convert("RGB")

    def caption_image(self, url):
        img = self.load_image(url)
        out = self.caption_pipeline(img)
        return out[0]["generated_text"]

    def caption_and_qa(self, url, qa_module):
        caption = self.caption_image(url)
        qa_res = qa_module.answer_questions(
            context=caption,
            questions=["What is shown in the image?"]
        )
        return caption, qa_res.get("What is shown in the image?", "[No answer]")

# ------------------------------------------------------------------------------
# 3. Symbolic Reasoning: Unification & Backward Chaining
# ------------------------------------------------------------------------------
class SymbolicModule:
    def __init__(self, rules):
        self.rules = [r.strip().rstrip('.') for r in rules]

    def _parse(self, expr):
        head, *body = expr.split(":-")
        pred, args = head.split("(")
        args = args[:-1].split(",") if args[:-1] else []
        body_preds = [b.strip() for b in body[0].split(",")] if body else []
        return pred.strip(), [a.strip() for a in args], body_preds

    def _unify(self, query, fact, theta=None):
        if theta is None:
            theta = {}
        p1, a1, _ = self._parse(query)
        p2, a2, _ = self._parse(fact)
        if p1 != p2 or len(a1) != len(a2):
            return None
        theta = theta.copy()
        for x, y in zip(a1, a2):
            if x[0].isupper():
                theta[x] = y
            elif y[0].isupper():
                theta[y] = x
            elif x != y:
                return None
        return theta

    def _backward_chain(self, query, theta=None, visited=None):
        if visited is None:
            visited = set()
        key = (query, frozenset((theta or {}).items()))
        if key in visited:
            return False
        visited.add(key)

        for rule in self.rules:
            pred, args, body = self._parse(rule)
            head = f"{pred}({','.join(args)})"
            subst = self._unify(query, head, theta)
            if subst is None:
                continue
            if not body:
                return True
            if all(self._backward_chain(b, subst, visited) for b in body):
                return True
        return False

    def query(self, q):
        return self._backward_chain(q)

# ------------------------------------------------------------------------------
# 4. Reinforcement Learning: Q-Learning with Experience Replay
# ------------------------------------------------------------------------------
class ReplayBuffer:
    def __init__(self, capacity=1000):
        self.buffer = deque(maxlen=capacity)

    def push(self, t): self.buffer.append(t)
    def sample(self, n): return random.sample(self.buffer, min(n, len(self.buffer)))

class QLearningAgent:
    def __init__(self, n_states, actions, lr=0.1, gamma=0.9, epsilon=0.1,
                 buffer_size=1000, batch_size=32):
        self.n_states = n_states
        self.actions = actions
        self.lr, self.gamma, self.epsilon = lr, gamma, epsilon
        self.q_table = {}
        self.replay = ReplayBuffer(buffer_size)
        self.batch_size = batch_size

    def _encode(self, s):
        return tuple(np.eye(self.n_states)[s].tolist())

    def _ensure(self, s):
        if s not in self.q_table:
            self.q_table[s] = {a: 0.0 for a in self.actions}

    def get_action(self, state):
        s = self._encode(state)
        self._ensure(s)
        if random.random() < self.epsilon:
            return random.choice(self.actions)
        return max(self.q_table[s], key=self.q_table[s].get)

    def update(self, s, a, r, ns):
        self.replay.push((s, a, r, ns))
        batch = self.replay.sample(self.batch_size)
        for s0, a0, r0, s1 in batch:
            e0, e1 = self._encode(s0), self._encode(s1)
            self._ensure(e0); self._ensure(e1)
            q0 = self.q_table[e0][a0]
            self.q_table[e0][a0] = q0 + self.lr * (
                r0 + self.gamma * max(self.q_table[e1].values()) - q0
            )

# ------------------------------------------------------------------------------
# 5. Unified AI System
# ------------------------------------------------------------------------------
class UnifiedAISystem:
    def __init__(self):
        self.qa = QAModule()
        self.vision = VisionModule()
        self.symbolic = SymbolicModule([
            "mortal(X) :- human(X)",
            "human(Socrates)."
        ])
        self.env = gym.make("FrozenLake-v1", is_slippery=False)
        n = self.env.observation_space.n
        self.rl = QLearningAgent(n_states=n, actions=list(range(self.env.action_space.n)))

    def run_nlp(self):
        ctx = ("France is a country in Western Europe. "
               "Its capital, Paris, is known for art and culture.")
        out = self.qa.answer_questions(ctx, [
            "What is the capital of France?",
            "Where is France located?"
        ])
        for q, a in out.items():
            print(f"NLP Q&A — {q} → {a}")

    def run_vision(self):
        url = (
            "https://upload.wikimedia.org/wikipedia/commons/thumb/a/a8/"
            "Tour_Eiffel_Wikimedia_Commons.jpg/"
            "320px-Tour_Eiffel_Wikimedia_Commons.jpg"
        )
        cap, ans = self.vision.caption_and_qa(url, self.qa)
        print(f"Vision — Caption: {cap}")
        print(f"Vision+NLP — What is shown? → {ans}")

    def run_symbolic(self):
        q = "mortal(Socrates)"
        print(f"Symbolic — Query: {q} → {self.symbolic.query(q)}")

    def run_rl(self, episodes=100, steps=50):
        rewards = []
        for _ in range(episodes):
            state = self.env.reset()
            total = 0
            for __ in range(steps):
                action = self.rl.get_action(state)
                step_out = self.env.step(action)
                # Handle both 4-tuple and 5-tuple APIs
                if len(step_out) == 5:
                    next_state, reward, terminated, truncated, _ = step_out
                    done = terminated or truncated
                else:
                    next_state, reward, done, _ = step_out

                self.rl.update(state, action, reward, next_state)
                total += reward
                state = next_state
                if done:
                    break
            rewards.append(total)
        avg = np.mean(rewards)
        print(f"RL — Average Reward over {episodes} episodes: {avg:.2f}")

    def run_all(self):
        print("\n--- NLP Module ---")
        self.run_nlp()
        print("\n--- Vision Module ---")
        self.run_vision()
        print("\n--- Symbolic Module ---")
        self.run_symbolic()
        print("\n--- RL Module ---")
        self.run_rl()

if __name__ == "__main__":
    UnifiedAISystem().run_all()